In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
# ۱. تهیه دیتاست بسیار ساده (Corpus)
sentences = [
    "من خیلی خوشحال هستم", "این عالی است", "خیلی خوب بود",
    "من خیلی غمگین هستم", "این بد است", "اصلا خوب نبود"
]
labels = [1, 1, 1, 0, 0, 0]  # 1: مثبت، 0: منفی

In [ ]:
# ۲. توکنایز کردن و آماده‌سازی ورودی
vocab = sorted(list(set(" ".join(sentences).split())))
word_to_id = {word: i + 1 for i, word in enumerate(vocab)} # 0 برای padding
vocab_size = len(word_to_id) + 1
maxlen = 5

def encode_sentences(data):
    encoded = []
    for s in data:
        ids = [word_to_id[w] for w in s.split()]
        encoded.append(ids + [0] * (maxlen - len(ids))) # Padding
    return np.array(encoded)

x_train = encode_sentences(sentences)
y_train = np.array(labels)

In [ ]:
# ۳. لایه Positional Encoding (به صورت کاستوم)
class PositionalEncoding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(PositionalEncoding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions

In [ ]:
# ۴. ساختار اصلی مدل Only-Encoder
def build_transformer_encoder(vocab_size, maxlen, embed_dim, num_heads, ff_dim):
    inputs = layers.Input(shape=(maxlen,))

    # الف) Input Embedding + Positional Encoding
    x = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
    x = PositionalEncoding(maxlen, embed_dim)(x)

    # ب) Multi-Head Attention
    # چون Encoder است، ماسک نداریم (همه کلمات همدیگر را می‌بینند)
    attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)

    # ج) Add & Norm (اولین بار)
    x = layers.LayerNormalization(epsilon=1e-6)(x + attention_output)

    # د) Feed Forward Network
    ffn_output = layers.Dense(ff_dim, activation="relu")(x)
    ffn_output = layers.Dense(embed_dim)(ffn_output)

    # هـ) Add & Norm (دومین بار)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ffn_output)

    # و) Pooling و خروجی نهایی (Linear + Softmax)
    # برای کارهای دسته‌بندی، معمولاً میانگین بردارها را می‌گیریم
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(2, activation="softmax")(x)

    return models.Model(inputs=inputs, outputs=outputs)

In [ ]:
# ۵. ساخت و کامپایل مدل
model = build_transformer_encoder(vocab_size, maxlen, embed_dim=32, num_heads=2, ff_dim=64)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# ۶. آموزش برای ۱۰۰ اپوک
print("شروع آموزش...")
history = model.fit(x_train, y_train, epochs=100)
print("آموزش تمام شد. آخرین دقت:", history.history['accuracy'][-1])

شروع آموزش...
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 0.0012
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 1.0000 - loss: 0.0012
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 1.0000 - loss: 0.0011
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - accuracy: 1.0000 - loss: 0.0011
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 1.0000 - loss: 0.0011
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step - accuracy: 1.0000 - loss: 0.0011
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 1.0000 - loss: 0.0011
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 1.0000 - loss: 0.0010
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 1.0000 - loss: 0.0010
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 1.0000 - loss: 0.0010
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 1.0000 - loss: 9.8625e-04
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/ste

In [ ]:
# ۷. تست مدل با جمله جدید
test_sentence = "این خوب نیست. خوشحال نیستم"
# پیش‌پردازش سریع تست
test_ids = [word_to_id.get(w, 0) for w in test_sentence.split()]
test_ids = test_ids + [0] * (maxlen - len(test_ids))
prediction = model.predict(np.array([test_ids]))

print(f"جمله: {test_sentence}")
print(f"احتمال (منفی، مثبت): {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
جمله: این خوب نیست. خوشحال نیستم
احتمال (منفی، مثبت): [[3.5311375e-04 9.9964690e-01]]
